In [5]:
import numpy as np
import pandas as pd
import re
from utilities import extract_data

In [11]:
#Creating initial dataframe
initial_data = pd.DataFrame(extract_data("accesslogs.log"))

In [13]:
#Saving parsed web logs
initial_data.to_csv('parsed.csv',index=False)